In [1]:
# import the necessary packages
from imutils import paths
import cv2
import os
import io
import re
import base64
from PIL import Image
import selenium
import requests
import time
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [2]:
# Download more than 500 images of any given parameter
def download_extended_urls(param):
    google_url = "https://www.google.com/imghp"
    driver = webdriver.Chrome()
    try:
        driver.get(google_url)
        #browser.visit(google_url)
        que=driver.find_element_by_xpath("//input[@name='q']")
        que.send_keys(param)
        que.send_keys(Keys.RETURN)       
        element = driver.find_element_by_tag_name("body")
        # Scroll down
        for i in range(60):
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.3)
        try:
            driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
            for i in range(100):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)  # bot id protection
        except:
            for i in range(30):
                element.send_keys(Keys.PAGE_DOWN)
                time.sleep(0.3)  # bot id protection

        print("Reached end of Page.")
        time.sleep(0.5)

        source = driver.page_source #page source
        #close the browser
        driver.close()
    except Error as e:
        print(e)
        
    url_list = []
    soup = bs(source, "html.parser")
    img = soup.find_all("img", class_='rg_i Q4LuWd tx8vtf')
    for i in img:
        try:
            url_list.append(i['src'])
        except:
            pass
    with open(f'urls_for_training/{param}.txt', 'w') as filehandle:
        for listitem in url_list:
            filehandle.write('%s\n' % listitem)
    return url_list


In [3]:
def download_img_from_url(param):
    # grab the list of URLs then initialize the
    # total number of images downloaded thus far
    total = 0
    # loop the URLs
    for url in download_extended_urls(param):#[:70]:
        try:
            r = requests.get(url, timeout=60)
            result_file = 'result_file'

            with open(result_file, 'wb') as file_handler:
                file_handler.write(r.content)
            try:
                os.mkdir(f"image/{param}/")
            except:
                pass
            p = os.path.join("image", param, "{}.jpg".format(str(total).zfill(8)))
            Image.open(result_file).save(p)
            os.remove(result_file)

            # update the counter
            print("[INFO] downloaded: {}".format(p))
        except:
            try:       
                z = re.findall(f'.+base64,(.+)',url)[0]
                p = os.path.join("image", param, "{}.jpg".format(str(total).zfill(8)))
                im = Image.open(io.BytesIO(base64.b64decode(z))).save(p)
            # handle if any exceptions are thrown during the download process
            except:
                print("[INFO] error downloading {}...skipping".format(p))
        total += 1
    # loop over the image paths we just downloaded
    for imagePath in paths.list_images(os.path.join("image", param)):
    # initialize if the image should be deleted or not
        delete = False

        # try to load the image
        try:
            image = cv2.imread(imagePath)
            # if the image is `None` then we could not properly load it
            # from disk, so delete it
            if image is None:
                print("None")
                delete = True
        # if OpenCV cannot load the image then the image is likely
        # corrupt so we should delete it
        except:
            print("Except")
            delete = True

        # check to see if the image should be deleted
        if delete:
            print("[INFO] deleting {}".format(imagePath))
            os.remove(imagePath)
    return

In [4]:
kitchen = "Silverware organizer, Plates, Coffee mugs, Tongs, Spatulas, Ladles and Skimmer, Whisks,\
    Utensil holder, Cutting board, Pots and Pans, Baking sheets, Tupperware,\
    Bottle Opener & Corkscrew, Measuring Cups, spoons, Toaster oven, Microwave, Coffee Maker,\
    Coffee Grinder, Kettle, Blender, Colander, Salad spinner, Trash can, Trash bags, Step stool,\
    Magnetic hooks, Sponges, Dish soap, Drying rack, Dish towels, Napkins, Paper Towels,\
    Aluminum foil, wax paper, plastic wrap, Parchment paper, Plastic bags, Ice cube trays,\
    Can opener, Oven mits, Cooling mat, Rags, Paper towels, Windex, Clorox wipes, Stove cleaner, Rubber Gloves"



coco = "bottle, wine glass, cup, fork, knife, spoon, bowl, diningtable, microwave, oven, toaster, \
       sink, refrigerator, scissors"

In [ ]:
param = "dog"
google_url = "https://www.google.com/imghp"
ChromeOptions = webdriver.ChromeOptions()
ChromeOptions.set_headless()
driver = webdriver.Chrome(chrome_options=ChromeOptions)
actionChains = ActionChains(driver)
try:
    driver.get(google_url)
    #browser.visit(google_url)
    que=driver.find_element_by_xpath("//input[@name='q']")
    que.send_keys(param)
    que.send_keys(Keys.RETURN)       
    element = driver.find_element_by_tag_name("body")
    # Scroll down
#     for i in range(5):
#         element.send_keys(Keys.PAGE_DOWN)
#         time.sleep(0.3)
#     try:
#         driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input').click()
#         for i in range(5):
#             element.send_keys(Keys.PAGE_DOWN)
#             time.sleep(0.3)  # bot id protection
#     except:
#         for i in range(5):
#             element.send_keys(Keys.PAGE_DOWN)
#             time.sleep(0.3)  # bot id protection
    driver.find_element_by_xpath('/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div/div[1]/div[1]/div[1]/a[1]/div[1]/img').click()
    #img = driver.find_element_by_xpath('/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/div/div[1]/div[1]/div/div[2]/a/img')
    
    img = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, '/html/body/div[2]/c-wiz/div[3]/div[2]/div[3]/div/div/div[3]/div[2]/div/div[1]/div[1]/div/div[2]/a/img')))
    src = img.get_attribute('src')
    print(src)
    print("Reached end of Page.")
    time.sleep(0.5)

    source = driver.page_source #page source
    soup = bs(source, "html.parser")
    #close the browser
    #driver.close()
except Error as e:
    print(e)

In [ ]:
all_images = driver.execute_script("return document.getElementsByTagName('img')")
for image in range(0, len(all_images)):
    print(driver.execute_script("return document.getElementsByTagName('img')[{}].src".format(image)))

In [196]:
z = re.findall(f'.+base64,(.+)',src)[0]
#p = os.path.join("image", param, "{}.jpg".format(str(total).zfill(8)))
im = Image.open(io.BytesIO(base64.b64decode(z)))
im.size

(300, 168)

In [24]:
Image.open("doggy.jpg").size

(300, 168)

In [62]:
Image.open("55043370.jpg").size

(257, 257)

In [ ]:
url_list = []
soup = bs(source, "html.parser")
img = soup.find_all("img", class_='rg_i Q4LuWd tx8vtf')
for i in img:
    try:
        url_list.append(i['src'])
    except:
        pass
with open(f'urls_for_training/{param}.txt', 'w') as filehandle:
    for listitem in url_list:
        filehandle.write('%s\n' % listitem)

In [ ]:
no_pages = 5
query = 'pots'

## Scrape Amazon

In [563]:
def get_amazon_data(query,no_of_pages): 
    def get_data(query,no_of_pages):
        amazon_dict = {
                        "featured": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "products_img":[],
                            "products_url":[],
                            "prime_delivery":[]
                        }, 
                        "low_to_high": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "products_img":[],
                            "products_url":[],
                            "prime_delivery":[]
                        },
                        "high_to_low": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "products_img":[],
                            "products_url":[],
                            "prime_delivery":[]
                        }, 
                        "average_customer_review": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "products_img":[],
                            "products_url":[],
                            "prime_delivery":[]
                        },
                        "newest_arrival": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "products_img":[],
                            "products_url":[],
                            "prime_delivery":[]
                        }
                      }

        executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
        browser = Browser("chrome", **executable_path, headless=True)
        for j in list(range(5)):
            for i in list(range(no_of_pages)):
                news_url = f"https://www.amazon.com/s?k={query}&page={i}"
                try:
                    browser.visit(news_url)
                    browser.find_by_xpath('//*[@id="a-autoid-0"]').click()
                    browser.find_by_xpath(f'//*[@id="s-result-sort-select_{str(j)}"]').click()
                    html = browser.html
                    soup = bs(html, "html.parser")

                    for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
                        name = d.find('span', attrs={'class':'a-size-base-plus a-color-base a-text-normal'})
                        price = d.find('span', attrs={'class':'a-offscreen'})
                        rating = d.find('span', attrs={'class':'a-icon-alt'})
                        prime = d.find('i', attrs={'class':'a-icon a-icon-prime a-icon-medium'})
                        img = d.find('img', attrs={'class':'s-image'})
                        url = d.find('a', attrs={'class':'a-link-normal a-text-normal'})

                        if name is not None and price is not None:
                            amazon_dict[list(amazon_dict.keys())[j]]["products_name"].append(name.text)
                            amazon_dict[list(amazon_dict.keys())[j]]["prices"].append(price.text)

                            if rating is not None:
                                amazon_dict[list(amazon_dict.keys())[j]]["ratings"].append(rating.text)
                            else:
                                amazon_dict[list(amazon_dict.keys())[j]]["ratings"].append("Missing")

                            if img is not None:
                                amazon_dict[list(amazon_dict.keys())[j]]["products_img"].append(img["src"])
                            else:
                                amazon_dict[list(amazon_dict.keys())[j]]["products_img"].append(" ")

                            if url is not None:
                                amazon_dict[list(amazon_dict.keys())[j]]["products_url"].append("https://amazon.com"+url["href"])
                            else:
                                amazon_dict[list(amazon_dict.keys())[j]]["products_url"].append(" ")

                            if prime is not None:
                                amazon_dict[list(amazon_dict.keys())[j]]["prime_delivery"].append("Prime Delivery")
                            else:
                                amazon_dict[list(amazon_dict.keys())[j]]["prime_delivery"].append("Not Prime Delivery")

                except Error as e:
                    print(e)
        browser.quit()
        return amazon_dict
    data = None
    str_error = None
    num_retries = 20
    for x in range(0, num_retries):  
        try:
            data = get_data(query,no_of_pages)
        except Error as e:
            pass

        if data["high_to_low"] == None:
            continue 
        else:
            break
    return data

# Scrape Walmart

In [565]:
def get_walmart_data(query,no_of_pages):
    def get_data(query,no_of_pages):
        drop_down = [
            "best_match--Best Match",
            "best_seller--Best Sellers",
            "price_low--Price: low to high",
            "price_high--Price: high to low",
            "rating_high--Highest Rating",
            "new--New"
        ]
        walmart_dict = {
                        "best_match": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        },
                        "best_sellers": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        },
                        "high_to_low": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        }, 
                        "low_to_high": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        },
                        "highest_rating": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        },
                        "newest_arrival": {
                            "products_name":[],
                            "prices": [],
                            "ratings":[],
                            "no_of_reviews":[],
                            "products_img":[],
                            "products_url":[],
                            "delivery_info":[]
                        }
                      }

        executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
        browser = Browser("chrome", **executable_path, headless=True)
        for i in list(range(1,no_of_pages+1)):
            url = f"https://www.walmart.com/search/?page={i}&query={query}"
            browser.visit(url)
            for j in list(range(6)):
                browser.execute_script("window.scrollTo(0, -document.body.scrollHeight);")
                browser.find_by_xpath('//*[@id="SearchContainer"]/div/div[2]/div/div[2]/div/div[2]/div[1]/div/select').click()
                browser.find_by_value(drop_down[j]).click()
                html = browser.html
                soup = bs(html, "html.parser")

                for d in soup.findAll('li'):
                    try:
                        name = d.find('div', attrs={'class':'search-result-product-title gridview'})
                        price = d.find('span', attrs={'class':'price display-inline-block arrange-fit price price-main'})\
                            .find('span', attrs={'class':'visuallyhidden'})
                        rating = d.find('span', attrs={'class':'seo-avg-rating'})
                        review = d.find('span', attrs={'class':'seo-review-count'})
                        delivery = d.find('div', attrs={'class':'search-result-product-shipping-details gridview'})
                        img = d.find('div', attrs={'class':'orientation-square'}).find("img")
                        url = d.find('a', attrs={'class':'product-title-link line-clamp line-clamp-2 truncate-title'})

                        if name.find("a").find("span") is not None and price is not None:
                            walmart_dict[list(walmart_dict.keys())[j]]["products_name"].append(name.find("a").find("span").text)
                            walmart_dict[list(walmart_dict.keys())[j]]["prices"].append(price.text)

                            if rating is not None:
                                walmart_dict[list(walmart_dict.keys())[j]]["ratings"].append(rating.text)
                            else:
                                walmart_dict[list(walmart_dict.keys())[j]]["ratings"].append("Missing Ratings")

                            if review is not None:
                                walmart_dict[list(walmart_dict.keys())[j]]["no_of_reviews"].append(rating.text)
                            else:
                                walmart_dict[list(walmart_dict.keys())[j]]["no_of_reviews"].append("Missing Reviews")

                            if img is not None:
                                walmart_dict[list(walmart_dict.keys())[j]]["products_img"].append(img["data-image-src"])
                            else:
                                walmart_dict[list(walmart_dict.keys())[j]]["products_img"].append(" ")

                            if url is not None:
                                walmart_dict[list(walmart_dict.keys())[j]]["products_url"].append("https://walmart.com"+url["href"])
                            else:
                                walmart_dict[list(walmart_dict.keys())[j]]["products_url"].append(" ")

                            if delivery is not None:
                                walmart_dict[list(walmart_dict.keys())[j]]["delivery_info"].append(delivery.text.replace("\xa0", " "))
                            else:
                                walmart_dict[list(walmart_dict.keys())[j]]["delivery_info"].append("Missing")

                    except:
                        pass
        browser.quit()
        return walmart_dict
    data = None
    str_error = None
    num_retries = 20
    for x in range(0, num_retries):  
        try:
            data = get_data(query,no_of_pages)
        except ElementClickInterceptedException as e:
            pass

        if data["high_to_low"] == None:
            continue 
        else:
            break
    return data


In [571]:
a = get_amazon_data('pencil',1)

In [572]:
b = get_walmart_data("pencil",1)

In [566]:
pd.DataFrame(a["high_to_low"])

,products_name,prices,ratings,products_img,products_url,prime_delivery
0,6th Graders Are #1-36 Sixth Grade Pencils D2009,"$3,263.34",Missing,https://m.media-amazon.com/images/I/31fjYXIHFW...,https://amazon.com/D2009-6th-Graders-Are-Penci...,Not Prime Delivery
1,Montblanc Meisterstuck Solitaire Malachite Hom...,"$2,900.00",5.0 out of 5 stars,https://m.media-amazon.com/images/I/91g7IAVrdZ...,https://amazon.com/Montblanc-Meisterstuck-Soli...,Not Prime Delivery
2,"10000 Pack, Personalized Custom Pencils, Round...","$2,349.00",4.8 out of 5 stars,https://m.media-amazon.com/images/I/61zoczn-w4...,https://amazon.com/Personalized-Incentive-Teac...,Not Prime Delivery
3,"5000 Pack, Military Personalized Custom Pencil...","$2,198.00",5.0 out of 5 stars,https://m.media-amazon.com/images/I/81sU3MI2gi...,https://amazon.com/Military-Personalized-Patri...,Not Prime Delivery
4,D2240 Virginia - 144 Virginia State Quick Fact...,"$2,100.02",Missing,https://m.media-amazon.com/images/I/31b+p0uQ8e...,https://amazon.com/D2240-Virginia-State-Quick-...,Not Prime Delivery
5,D2460 Crush The ISTEP - 144 Indiana Test Pencils,"$1,723.44",Missing,https://m.media-amazon.com/images/I/61P+rYUSkM...,https://amazon.com/D2460-Crush-ISTEP-Indiana-P...,Not Prime Delivery
6,NRS Natural Roughout Pencil Roll Team 14.5,"$1,701.72",Missing,https://m.media-amazon.com/images/I/01RmK+J4pJ...,https://amazon.com/NRS-Natural-Roughout-Pencil...,Not Prime Delivery
7,"5000 Pack, Personalized Custom Pencils, NEON R...","$1,349.00",5.0 out of 5 stars,https://m.media-amazon.com/images/I/71G-uImfiC...,https://amazon.com/Personalized-Incentive-Teac...,Not Prime Delivery
8,Montblanc Alexandre Dumas 1996 Limited Edition...,$999.00,Missing,https://m.media-amazon.com/images/I/01RmK+J4pJ...,https://amazon.com/Montblanc-Alexandre-Limited...,Not Prime Delivery
9,Mont Blanc Meisterstück Leonardo Sketch Pencil,$957.00,Missing,https://m.media-amazon.com/images/I/41s3J4BVQk...,https://amazon.com/Blanc-Meisterst%C3%BCck-Leo...,Not Prime Delivery


In [568]:
pd.DataFrame(b['high_to_low'])

,products_name,prices,ratings,no_of_reviews,products_img,products_url,delivery_info
0,Ticonderoga Soft Pencil - Yellow - Pre-Sharpen...,$6.33,4.7,4.7,https://i5.walmartimages.com/asr/3e2be000-2800...,https://walmart.com/ip/Ticonderoga-Soft-Pencil...,Free delivery on orders over $35
1,"Bic Xtra-Life Mechanical Pencils, Medium Point...",$4.54,4.6,4.6,https://i5.walmartimages.com/asr/3f8fc045-03b2...,https://walmart.com/ip/Bic-Xtra-Life-Mechanica...,Sold & shipped by Pharmapacks
2,"Ticonderoga Pencil, 24 Count, Unsharpened",$7.57,4.7,4.7,https://i5.walmartimages.com/asr/4dfceb90-e553...,https://walmart.com/ip/Ticonderoga-Pencil-24-C...,Sold & shipped by TheMarket
3,Bic 24+3 Bonus Mechanical Pencil 0.7mm,$4.48,4.7,4.7,https://i5.walmartimages.com/asr/a61922f0-a57f...,https://walmart.com/ip/Bic-24-3-Bonus-Mechanic...,Free delivery on orders over $35Free pickup today
4,"Crayola Erasable Twistables Colored Pencils, 1...",$5.76,4.5,4.5,https://i5.walmartimages.com/asr/b4ddf078-5ef6...,https://walmart.com/ip/Crayola-Erasable-Twista...,2-day deliveryon orders $35+
5,"BIC Xtra-Precision Mechanical Pencil, Metallic...",$4.93,4.3,4.3,https://i5.walmartimages.com/asr/e4250bd5-4397...,https://walmart.com/ip/BIC-Xtra-Precision-Mech...,Free pickup today
6,BiC Xtra Precision Mechanical Pencils #2 Fine ...,$4.74,4,4,https://i5.walmartimages.com/asr/d1bccc7c-c6ba...,https://walmart.com/ip/BiC-Xtra-Precision-Mech...,Sold & shipped by United Art and Education
7,"Paper Mate Clearpoint Mechanical Pencil Set, 0...",$4.46,4.4,4.4,https://i5.walmartimages.com/asr/016f894e-b6aa...,https://walmart.com/ip/Paper-Mate-Clearpoint-M...,Free delivery on orders over $35Free pickup today
8,"Crayola Erasable Colored Pencil Set, 24-Colors",$5.97,4.1,4.1,https://i5.walmartimages.com/asr/256a0c1f-f521...,https://walmart.com/ip/Crayola-Erasable-Colore...,Free pickup today
9,"BIC Mechanical Pencil Xtra Life, Black, 10+2 Pack",$19.95,4.9,4.9,https://i5.walmartimages.com/asr/5fd592bd-fd8d...,https://walmart.com/ip/BIC-Mechanical-Pencil-X...,Sold & shipped by Rocky Mountain TextbooksFree...


In [569]:
pd.DataFrame(a["low_to_high"])

,products_name,prices,ratings,products_img,products_url,prime_delivery
0,"DIXON No. 2 Yellow Pencils, Wood-Cased, Black ...",$3.48,4.4 out of 5 stars,https://m.media-amazon.com/images/I/71-tCs1ObM...,https://amazon.com/Dixon-Pencils-Wood-Cased-8-...,Prime Delivery
1,Cra-Z-Art No. 2 wood Pencils 8 Count,$3.79,Missing,https://m.media-amazon.com/images/I/71VvJWWCtU...,https://amazon.com/Cra-Z-Art-No-l%C3%A1pices-m...,Prime Delivery
2,General Pencil 557-HB-BP Generals Charcoal Pen...,$6.76,4.4 out of 5 stars,https://m.media-amazon.com/images/I/61vd0PBUOL...,https://amazon.com/General-Pencil-557-HB-BP-Ge...,Prime Delivery
3,"Lead Small, 12 Color Small Pencil Painting Pen...",$0.01,Missing,https://m.media-amazon.com/images/I/517PUpn3ms...,https://amazon.com/Pencil-Painting-Stationery-...,Not Prime Delivery
4,Dergo ☀Count Rainbow Pencils Black and White W...,$0.01,4.0 out of 5 stars,https://m.media-amazon.com/images/I/5168bLsXJi...,https://amazon.com/Dergo-%E2%98%80Count-Rainbo...,Not Prime Delivery
5,Paper Mate 74402 .7mm Write Brothers Mechanica...,$0.87,Missing,https://m.media-amazon.com/images/I/2148rHdDpd...,https://amazon.com/Paper-Mate-74402-Brothers-M...,Not Prime Delivery
6,Dergo Count Rainbow Pencils Black And White Wo...,$0.01,Missing,https://m.media-amazon.com/images/I/51PdnWM72M...,https://amazon.com/Dergo-Rainbow-Pencils-Schoo...,Not Prime Delivery
7,Dergo ☀ Iron Metal Mechanical Automatic Pencil...,$0.01,5.0 out of 5 stars,https://m.media-amazon.com/images/I/41SkilOpMd...,https://amazon.com/Dergo-Mechanical-Automatic-...,Not Prime Delivery
8,Dergo ☀ Iron Metal Mechanical Automatic Pencil...,$0.01,1.0 out of 5 stars,https://m.media-amazon.com/images/I/4117Ewizn3...,https://amazon.com/Dergo-Mechanical-Automatic-...,Not Prime Delivery
9,"5pc Pencils Set,Black and White Wood Set Rainb...",$0.01,Missing,https://m.media-amazon.com/images/I/517dq5nGg9...,https://amazon.com/l%C3%A1pices-madera-blanco-...,Not Prime Delivery


In [570]:
pd.DataFrame(b['low_to_high'])

,products_name,prices,ratings,no_of_reviews,products_img,products_url,delivery_info
0,"Paper Mate Write Bros. Mechanical Pencils, 0.7...",$1.00,1,1,https://i5.walmartimages.com/asr/04daf5d4-dcdb...,https://walmart.com/ip/Paper-Mate-Write-Bros-M...,Free pickup today
1,"BIC Velocity Pro Mechanical Pencil, Medium Poi...",$2.84,5,5,https://i5.walmartimages.com/asr/0b1e61be-79cc...,https://walmart.com/ip/BIC-Velocity-Pro-Mechan...,Free pickup today
2,Paper Mate Handwriting Triangular Mechanical P...,$3.24,5,5,https://i5.walmartimages.com/asr/365f30ae-51d2...,https://walmart.com/ip/Paper-Mate-Handwriting-...,
3,"Pentel Sharp Mechanical Pencil (0.9mm), Yellow...",$3.29,4.8,4.8,https://i5.walmartimages.com/asr/c3100de7-8f63...,https://walmart.com/ip/Pentel-Sharp-Mechanical...,Free delivery on orders over $35
4,"Paper Mate Write Bros. Mechanical Pencils, 0.7...",$3.46,4.8,4.8,https://i5.walmartimages.com/asr/d528cc8c-17cb...,https://walmart.com/ip/Paper-Mate-Write-Bros-M...,Sold & shipped by Pharmapacks
5,BiC Xtra Precision Mechanical Pencils #2 Fine ...,$4.74,4,4,https://i5.walmartimages.com/asr/d1bccc7c-c6ba...,https://walmart.com/ip/BiC-Xtra-Precision-Mech...,Sold & shipped by United Art and Education
6,"Bic Velocity Original Mechanical Pencil, Thick...",$3.64,4.8,4.8,https://i5.walmartimages.com/asr/3c00226b-c390...,https://walmart.com/ip/Bic-Velocity-Original-M...,Free pickup today
7,"Pentel Twist-Erase III Mechanical Pencil, (0.9...",$4.28,4.8,4.8,https://i5.walmartimages.com/asr/e06a3bbd-4a0f...,https://walmart.com/ip/Pentel-Twist-Erase-III-...,2-day deliveryon orders $35+
8,"Paper Mate Clearpoint Mechanical Pencil Set, 0...",$4.46,4.4,4.4,https://i5.walmartimages.com/asr/016f894e-b6aa...,https://walmart.com/ip/Paper-Mate-Clearpoint-M...,Free delivery on orders over $35Free pickup today
9,Bic 24+3 Bonus Mechanical Pencil 0.7mm,$4.48,4.7,4.7,https://i5.walmartimages.com/asr/a61922f0-a57f...,https://walmart.com/ip/Bic-24-3-Bonus-Mechanic...,Free delivery on orders over $35Free pickup today


In [ ]:
def get_data(query,no_of_pages):
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    browser = Browser("chrome", **executable_path, headless=True)
    for j in list(range(5)):
        for i in list(range(no_of_pages)):
            news_url = f"https://www.amazon.com/s?k={query}&page={i}"
            try:
                browser.visit(news_url)
                browser.find_by_xpath('//*[@id="a-autoid-0"]').click()
                browser.find_by_xpath(f'//*[@id="s-result-sort-select_{str(j)}"]').click()
                html = browser.html
                soup = bs(html, "html.parser")

                for d in soup.findAll('div', attrs={'class':'sg-col-4-of-24 sg-col-4-of-12 sg-col-4-of-36 s-result-item s-asin sg-col-4-of-28 sg-col-4-of-16 sg-col sg-col-4-of-20 sg-col-4-of-32'}):
                    url = d.find('a', attrs={'class':'a-link-normal a-text-normal'})

                    if name is not None and price is not None:
                        amazon_dict[list(amazon_dict.keys())[j]]["products_name"].append(name.text)
                        amazon_dict[list(amazon_dict.keys())[j]]["prices"].append(price.text)

                        if rating is not None:
                            amazon_dict[list(amazon_dict.keys())[j]]["ratings"].append(rating.text)
                        else:
                            amazon_dict[list(amazon_dict.keys())[j]]["ratings"].append("Missing")

                        if img is not None:
                            amazon_dict[list(amazon_dict.keys())[j]]["products_img"].append(img["src"])
                        else:
                            amazon_dict[list(amazon_dict.keys())[j]]["products_img"].append(" ")

                        if url is not None:
                            amazon_dict[list(amazon_dict.keys())[j]]["products_url"].append("https://amazon.com"+url["href"])
                        else:
                            amazon_dict[list(amazon_dict.keys())[j]]["products_url"].append(" ")

                        if prime is not None:
                            amazon_dict[list(amazon_dict.keys())[j]]["prime_delivery"].append("Prime Delivery")
                        else:
                            amazon_dict[list(amazon_dict.keys())[j]]["prime_delivery"].append("Not Prime Delivery")

            except Error as e:
                print(e)
    browser.quit()
    return amazon_dict

In [14]:
import get_cnn_data as get
get.get_amazon_data('pencil',1)

{'featured': {'products_name': ['TICONDEROGA My First Pencils, Wood-Cased #2 HB Soft, Pre-Sharpened with Eraser, Includes Bonus Sharpener, Yellow, 4-Pack (33309)',
   'Ticonderoga Pencils, Graphite #2, Yellow, 18-Pack',
   'AmazonBasics Pre-sharpened Wood Cased #2 HB Pencils, 150 Pack',
   'Dixon No. 2 Yellow Pencils, Wood-Cased, Black Core, 12-Count (14402)',
   'BIC Xtra-Sparkle Mechanical Pencil, Medium Point (0.7 mm), 24-Count',
   'TICONDEROGA Pencils, Wood-Cased, Unsharpened, Graphite #2 HB Soft, Yellow, 96-Pack (13872)',
   'BIC Velocity Original Mechanical Pencil, Medium Point (0.7mm), 4-Count',
   'BIC Xtra-Life Mechanical Pencil,  0.7mm, 10 Ct',
   'Paper Mate Clearpoint Mechanical Pencil, 0.7 mm, Black Barrel, Refillable, 4-pack',
   'Ticonderoga Pencils, Wood-Cased, Graphite #2 HB Soft, Black, 24-Pack (13926)',
   'Ticonderoga Noir Black Wood-Cased #2 Pencils, Holographic Design, 12-Count (13970)',
   'Crayola Erasable Colored Pencils, 10 Count, School Supplies',
   'BIC Xt

In [10]:
import tensorflow as tf
raw_images = []
images = os.listdir(os.getcwd() + '/uploads')
image_names = []
for image in images:
        image_names.append(image)
        image_file = os.getcwd() + '/uploads/' + image
        #img_raw = tf.image.decode_image(open(image_file, 'rb').read(), channels=3)
        img_raw = tf.io.decode_image(image_file, channels=3, dtype=tf.dtypes.uint8,expand_animations=True)
        raw_images.append(img_raw)
num = 0
print(image_file)

['dog.jpg', 'office.jpg']


In [ ]:
import tensorflow as tf
raw_images = []
images = os.listdir(os.getcwd() + '/uploads')
image_names = []
for image in images:
    for i in ["jpg","jpeg","png","bmp"]:
        if image.endswith(i):
            image_names.append(image)
            image_file = os.getcwd() + '/uploads/' + image
            img_raw = tf.image.decode_image(open(image_file, 'rb').read(), channels=3)
            raw_images.append(img_raw)
    num = 0
print(raw_images)